# Blog Post Series - AI Fairness 360: Mitigating Bias in Machine Learning Models

## Part 1: **Creating Dataset Objects in AI Fairness 360 and Exploring Fairness Metrics**


### Introduction to AI Fairness

Artificial Intelligence (AI) has revolutionized various sectors, from healthcare to finance, by providing powerful tools to analyze data and make informed decisions. However, as these AI systems become more integrated into society, ensuring that they operate fairly and without bias becomes increasingly important. Bias in AI can lead to unfair outcomes, which can have significant social and ethical implications.

To address this issue, IBM developed AI Fairness 360 (AIF360), an open-source toolkit designed to help developers detect and mitigate bias in machine learning models. 

In this lesson, we will explore what AI Fairness 360 is, why fairness in AI is crucial, and how to start using the toolkit to create more equitable AI systems.




### Learning Objectives

By the end of this blog post, you will be able to:
- Understand the concept of bias in AI and its implications.
- Learn how to use AI Fairness 360 (AIF360) to detect and mitigate bias.
- Create dataset objects using AIF360's `BinaryLabelDataset` and `StandardDataset` classes.
- Utilize fairness metrics to evaluate bias in datasets and models.



### Prerequisites

Before we begin, ensure you have the following installed:
- Python 3.x
- Pandas
- AI Fairness 360





### Understanding Bias in AI

As we delve into the topic of fairness in AI, it's essential to first understand what bias is and how it can manifest in machine learning systems.

**Types of Bias**
Bias in AI can stem from various sources and can be broadly categorized into three types:

1. **Data Bias**: This occurs when the training data used to build the model is not representative of the real-world population. For example, if a facial recognition system is trained predominantly on images of lighter-skinned individuals, it may perform poorly on individuals with darker skin tones.

2. **Algorithmic Bias**: This arises when the algorithms used in the AI system inadvertently amplify existing biases in the data. Even if the training data is balanced, the way the algorithm processes this data can still introduce bias.

3. **Societal Bias**: This type of bias reflects broader societal prejudices and inequalities. AI systems, being products of human society, can inherit and perpetuate these biases if not carefully designed and monitored.

**Impact of Bias**
The impact of biased AI systems can be profound and far-reaching. Biased models can lead to discriminatory practices in areas such as hiring, lending, law enforcement, and healthcare. For instance, a biased hiring algorithm might unfairly favor candidates from certain demographic groups, leading to unequal employment opportunities.

Real-world examples of biased AI systems highlight the urgency of addressing this issue. For instance, a study found that a widely used healthcare algorithm disproportionately favored white patients over black patients when determining eligibility for certain health programs, leading to disparities in care.



### Introduction to AI Fairness 360

AI Fairness 360 (AIF360) is an open-source toolkit developed by IBM to address the challenge of bias in machine learning models. The toolkit provides a comprehensive suite of metrics to test for biases and algorithms to mitigate them.

**Toolkit Overview**
AIF360 includes:

- **Datasets**: A variety of datasets commonly used in fairness research, such as the Adult Income dataset and the COMPAS dataset.
- **Metrics**: A collection of fairness metrics to evaluate bias in datasets and models, including disparate impact, statistical parity, and equal opportunity difference.
- **Algorithms**: Several bias mitigation algorithms, such as reweighing, prejudice remover, and adversarial debiasing.

**Installation**
To get started with AI Fairness 360, you first need to install the toolkit. You can install it using pip:



You can install the base components AI Fairness 360 using pip:
```bash
pip install aif360
```

To install all optional dependecies use:
```bash
pip install 'aif360[all]'
```

For more information see https://aif360.readthedocs.io/en/latest/Getting%20Started.html#installation.


In the following sections, we will walk through a practical example using AIF360 to detect and mitigate bias in a machine learning model.



---

### Practical Example: Detecting and Mitigating Bias




In this example, we will use the Recidivism for Offenders Released from Prison dataset [directly from the IOWA Open Data Portal](https://data.iowa.gov/Correctional-System/Iowa-Prison-Recidivism-Status/akzb-ddk8/about_data), which provides information on individuals released from prison and their likelihood of reoffending. This dataset is suitable for studying fairness as it contains various demographic features that can be analyzed for bias.


**Dataset Construction**
To use AIFairness 360, the data must be converted into a proprietary AIF360 dataset object. The requirements for the Dataset objects are as follows:
1. Categorical Features encoded numerically (Not One hot encoded).
2. No null values.
3. Defined **Protected attributes** (e.g. Race, Sex).
    - **Definition**: Attributes that refer to characteristics of individuals that are legally protected against discrimination. These can include race, gender, age, religion, disability status, and other similar attributes.
4. For each protected attribute, defined Privileged groups (e.g. Male or White) and Unpriviledged Groups (e.g. Female, Non-white). 
    - **Privileged Groups**: Subsets of individuals within a dataset that have historically been favored or have had advantages in societal contexts based on certain protected attributes.
    - **Unprivileged Groups**: Subsets of individuals within a dataset that have historically been disadvantaged or have faced discrimination based on certain protected attributes.


### Introduction to AIF360 Datasets


In order to use AIF360, we must convert our DataFrames into one of two classes from AIF360: the `BinaryLabelDataset` and the `StandardDataset`.

In this blog post, we will focus on creating dataset objects using both the `BinaryLabelDataset` and `StandardDataset` classes provided by AIF360. 

Additionally, we will begin to explore some key fairness metrics to understand how bias can be measured and addressed.



#### Loading the Dataset as DataFrame First


In [ ]:

import pandas as pd
import numpy as np
pd.set_option("display.max_columns", 100)

# Load the data 
df = pd.read_csv("blog_post/data/Iowa_Prison_Recidivism_Status_20240724.csv", 
                 index_col=0, usecols=range(0, 23-7))   

## Quick Conversion of Dtypes for Clean Data
df = df.convert_dtypes(convert_string=False)

# Drop unnecessary columns
drop_cols = ['Supervising Unit','Supervision Start Date','Supervision End Date'] + [c for c in df.columns if 'Year' in c]
df = df.drop(columns=drop_cols)
df.info()
df.head()




#### Encoding Protected Attributes Features as Integers

To work with AIF360, we need to encode the categorical features numerically. In this example, we'll focus on the 'race' attribute as a protected attribute.


In [ ]:
# Preview unique classes for protected attributes
df['Race'].unique(), df['Sex'].unique()

While I have not yet found an official statement on whether protected attribute columns are able to be used with the metrics we will be discussing in this post, this author was unable to get mult-classes protected attributes to work. Therefore, for now, we have converted the Race column into a binary column where 0 represents being White and 1 represents Non-White. 

In [ ]:
# Encode the 'race' column
# race_map = {'White': 0, 'Black': 1, 'Hispanic': 2, 'Asian or Pacific Islander': 3,
            # 'American Indian or Alaska Native': 4, 'Unknown': 5, 'Other':6}
race_map = {'White': 0, 'Black': 1, 'Hispanic': 1, 'Asian or Pacific Islander': 1,
            'American Indian or Alaska Native': 1, 'Unknown': 1, 'Other':1}
df['Race'] = df['Race'].map(race_map)

# Encode the "Sex" column
sex_map = {"Male": 0, "Female":1}
df['Sex'] = df['Sex'].map(sex_map)

df['Race'].unique(), df['Sex'].unique()

We will be using both string and integer labels for our groups throughout this example. Therefore, we will save the dictionary maps together for later.

In [ ]:
## Saving the protcted attribute mapping dictionaries to a dict
protected_attribute_maps = {"Race":race_map,
                            "Sex":sex_map}

### Preprocessing for Dataset Object Creation

The AI Fairness 360 (AIF360) toolkit provides different dataset classes to handle various types of data structures and use cases. Two commonly used classes are `BinaryLabelDataset` and `StandardDataset`. 

Both classes require that:
- The protected attributes (i.e. Race and Sex) and target (Reincarcerated) must be encoded as integers.
- All features must be numerically encoded (Ohe Hot Encoded categorical features, Boolean features as integers)
- No null values are present.

Therefore we will prepare the data as if we were preparing it for machine learning. However, we will not produce a train-test-split.

In [ ]:
# Impute NaNs with most frequent value
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline

# Enable pandas dataframe output
from sklearn import set_config
set_config(transform_output='pandas')


In [ ]:

# Categorical Pipeline
cat_cols = df.select_dtypes(include='object').columns
cat_imputer = SimpleImputer(strategy='most_frequent')
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
cat_pipe = make_pipeline(cat_imputer, ohe)


# Numeric Pipeline
num_cols = df.select_dtypes(include='number').columns
num_imputer = SimpleImputer(strategy='mean')
num_pipe = make_pipeline(num_imputer)


# Convert Boolean Columns to Integers
bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)


# Create the column Transformer
preprocessor = ColumnTransformer(transformers=[('cat', cat_pipe, cat_cols),
                                               ('num', num_pipe, num_cols)],
                                 remainder='passthrough',
                                 verbose_feature_names_out=False)

preprocessor

In [ ]:
# Fit and Transform the data
final_df = preprocessor.fit_transform(df)
final_df

### Defining Protected Attributes and Priviledged vs. Unpriveldged Groups

Before we can convert our preprocessed data to a BinaryDataset or StandardDataset, we need to take a moment to fully map the protected attributes and for each attribute, we must define priviledged groups and unpriviledged groups.

If you use one of the Built-In datasets, it comes with a data dictionary. If we construct a similar data dictionary, we can leverage example code from official AIF360 tutorials to convert our data_dict into the correctly formattd group variables.

#### Examining the Data Dict of the Tutorial Dataset

- Tutorial: Medical Expendature: https://github.com/Trusted-AI/AIF360/blob/main/examples/tutorial_medical_expenditure.ipynb
- See the notebook linked above for instructions on running this example locally. 
Note that we will only briefly analyze the contents of this tutorial example, so you can safely skip over these steps when working through this example on your own.

In [ ]:
## Tutorial Code source:  https://github.com/Trusted-AI/AIF360/blob/main/examples/tutorial_medical_expenditure.ipynb

# Datasets
from aif360.datasets import MEPSDataset19
from aif360.datasets import MEPSDataset20
from aif360.datasets import MEPSDataset21

# dataset = MEPSDataset19()#.split([0.5, 0.8], shuffle=True)

(dataset_orig_panel19_train,
 dataset_orig_panel19_val,
 dataset_orig_panel19_test) = MEPSDataset19().split([0.5, 0.8], shuffle=True)

# Numeric index for sensitive/protected attribute
sens_ind = 0

# Name of the sensitive/protected attribute
sens_attr = dataset_orig_panel19_train.protected_attribute_names[sens_ind]

# List comp to create a list of dictionaries for unprivileged and privileged groups
unprivileged_groups = [{sens_attr: v} for v in
                       dataset_orig_panel19_train.unprivileged_protected_attributes[sens_ind]]
privileged_groups = [{sens_attr: v} for v in
                     dataset_orig_panel19_train.privileged_protected_attributes[sens_ind]]

# Preview the group vars
print(f"{unprivileged_groups = }")
print(f"{privileged_groups = }")

Notice that the dataset objects already had information about the `protected_attribute_names` and the integer values for the encoded protected attributes (`privileged_protected_attributes` `unprivileged_protected_attributes` ).

The tutorial used a list comprehension to convert the arrays of group integer values into a list of dictionaries, with the `[{Protected Attribute Name:Integer Value}]`.


The tutorial then demonstrates instantiated a  BinaryLabelDatasetMetric with the dataset and the list of priviledged group dictionaries.

In [ ]:
# Fairnes metrics & Explainers
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.explainers import MetricTextExplainer


# Metrics for the tutorial dataset
metric_orig_panel19_train = BinaryLabelDatasetMetric(
        dataset_orig_panel19_train,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)


The tutorial then instanstiates a MetricTextExplainer to facilitate understanding of the metrics.  The first example metric, which will discuss more in-depth later, is Disparate Impact. 

In [ ]:

# Explainer to describe the fairness metrics
explainer_orig_panel19_train = MetricTextExplainer(metric_orig_panel19_train)
# Displaying disparate_impact
print(explainer_orig_panel19_train.disparate_impact())

Now, to begin analyzing the metrics of our Prisoner Recividism dataset, we will first want to declare the protected attributes, and priviledged and unpriviledged group numbers.

### Mimicking the Tutorial's Data  Dictionary

#### Extracting and Inspecting the Tutorial Data Dictionary 


If we convert one of the example datasets to a dataframe using the `.convert_to_dataframe()` method, we will get the dataframe, as well as a data dict that stores the information about the protected attributes and priviledge groups.

In [ ]:
# Converting the example dataset to a dataframe and data dictionary 
example_df, example_data_dict = dataset_orig_panel19_train.convert_to_dataframe()#)
# display(example_df.head(3))

# Exploring the contents of the data dictionary
print("Data Dict Keys:")
[print(f"- {k}") for k in example_data_dict.keys()];

In [ ]:
# Target label var stored in list of strings - Only 1 target column
example_data_dict['label_names']

In [ ]:
# List of protected attribute column names
example_data_dict['protected_attribute_names']

For each protected attribute, the data dictionary has an array of the group values that are priviledged (believed to have an unfair advantage) and unpriviledged. 


In [ ]:
# List of privileged values for each protected attribute
display(example_data_dict['privileged_protected_attributes'])

# List of unprivileged values for each protected attribute
display(example_data_dict['unprivileged_protected_attributes'])


> Take note that, in this example, we have only 1 protected attribute. 

For the priviledged/unpriviledged groups, we have a list, with 1 array for each of our protected attributes, but in this case we only have one so its a list of a single array.

#### Creating our own data_dictionary



Let's create a new data dictionary based on our Iowa Prisoner Recidivism dataset. In this dataset, we have more than 1 protected attribute, which increases the complexity of our code.

In [ ]:
example_data_dict.keys()

Since all of the representations of the groups need to be converted to lists of integers, we will save our integer-group mapping dictionaries to a final dictionary under the key for the corresodning atribute.

We will also save a dictionary with a key for each protected attribute and the **names** of the priviledged groups for each protected attribute.

In [ ]:
## Saving the protcted attribute mapping dictionaries to a dict

# Commented Out: previously saved above
# protected_attribute_maps = {"Race":race_map,
#                             "Sex":sex_map}

print("- protected_attribute_maps:")
display(protected_attribute_maps)

# Manually defining the list of privileged group NAMES to a dict
prot_attrs_priv_group_names = {'Race':["White"],
                               "Sex":["Male"]}

print("- prot_attrs_priv_group_names:")
prot_attrs_priv_group_names

Let's review the structure of the example data dictionary before we reproduce it for our prisoner recidivism data.

In [ ]:
# Revieing the structure of 
example_data_dict.keys()

In [ ]:
# Creating a data dictionary like the one used in AIF360's medical expenditure example
target = 'Reincarcerated'

# Saving the feature names as a list
feature_names = final_df.drop(columns=target).columns
protected_attributes_names = list(protected_attribute_maps.keys())

# Starting our own data dictionary
DATA_DICT = dict(feature_names=feature_names, # list of feature columns
                 label_names = [target], # list with just the target label column
                 protected_attributes_names = protected_attributes_names, # list of protected attribute columns
                 
                # Saving information about each row (not used )
                #  instance_names = final_df.index, # list of each row's name
                #  instance_weights = np.ones_like(final_df.index), # list of each row's weight (1.0 for now
                
                
                # List of arrays of priviledged group numbers: will be filled in later
                privileged_protected_attributes = [], 
                # List of arrays of unpriviledged group numers: Will be filled in later
                unprivileged_protected_attributes = [], 
)
                 
DATA_DICT.keys()


### For Loop: Accounting for More than 1 Protected Attribute per Dataset

We will need to produce the group information for every protected attribute in the dataset. In this case, the proteted attributes are Race and Sex.
Before we construct a loop to handle both protected attributes, let's manually slice out the first attribute and figure out our workflow.

In [ ]:
# Numeric index of current protected attribute
attr_idx = 0

# slice the name of the current protected attribute
attr_name = protected_attributes_names[attr_idx]

# Get the mapping for the current protected attribute
attr_map = protected_attribute_maps[attr_name]
attr_map

In [ ]:
# # Get the privileged group names for the current protected attribute
priv_group_names = prot_attrs_priv_group_names[attr_name]
priv_group_names

In [ ]:
# Get the privileged values for the current protected attribute
priviledged_group_nums = np.array([attr_map[pg] for pg in priv_group_names])
priviledged_group_nums

In [ ]:
# Get the unprivileged values for the current protected attribute
unpriviledged_group_nums = np.array([v for v in attr_map.values() if v not in priviledged_group_nums])
unpriviledged_group_nums

Now that we've been able to save the numeric valus for the privileged and unprivileged groups, we can now save them to the data dictionary.

In [ ]:
# Saving the privileged and unprivileged group values to the data dictionary
DATA_DICT['privileged_protected_attributes'].append(priviledged_group_nums)
DATA_DICT['unprivileged_protected_attributes'].append(unpriviledged_group_nums)

In [ ]:
# # Get current attributes privileged groups
# current_attr_priv_group_names = prot_attrs_priv_group_names[attr_name]
# current_attr_priv_group_names

In [ ]:
# # Save unique groups for the current attribute
# unique_groups = current_attr_map_dict.values()
# unique_groups

### Full Data Dictionary Creation Loop: Putting it All Together



We can use our mapping dictionaries to set the integer values for priviledged and unpriviledged groups. 
We know for Race, the priviledged group is "White" and all other values are unpriviledged groups.

In [ ]:
## Combining the full workflow into a loop

# Creating a data dictionary like the one used in AIF360's medical expenditure example
target = 'Reincarcerated'
# Save the target label names as a list


# Saving the feature names as a list
feature_names = final_df.drop(columns=target).columns
protected_attributes_names = list(protected_attribute_maps.keys())

# Starting our own data dictionary
DATA_DICT = dict(feature_names=feature_names, # list of feature columns
                 label_names = [target], # list with just the target label column
                 protected_attributes_names = protected_attributes_names, # list of protected attribute columns
                 
                # Saving information about each row (not used )
                #  instance_names = final_df.index, # list of each row's name
                #  instance_weights = np.ones_like(final_df.index), # list of each row's weight (1.0 for now
                
                
                # List of arrays of priviledged group numbers: will be filled in later
                privileged_protected_attributes = [], 
                # List of arrays of unpriviledged group numers: Will be filled in later
                unprivileged_protected_attributes = [], 
)


# Loop through the protected attributes
for attr_idx, attr_name in enumerate(protected_attributes_names):
    
    # Get the mapping for the current protected attribute
    attr_map = protected_attribute_maps[attr_name]
    
    # Get the privileged group names for the current protected attribute
    priv_group_names = prot_attrs_priv_group_names[attr_name]
    
    # Get the privileged values for the current protected attribute
    priv_vals = [attr_map[pg] for pg in priv_group_names]
    
    # Get the unprivileged values for the current protected attribute
    unpriv_vals = [v for v in attr_map.values() if v not in priv_vals]
    
    # Add the privileged and unprivileged values to the data dictionary
    DATA_DICT['privileged_protected_attributes'].append(np.array(priv_vals))
    DATA_DICT['unprivileged_protected_attributes'].append(np.array(unpriv_vals))
    
print(f"{DATA_DICT['privileged_protected_attributes']=}")
print(f"{DATA_DICT['unprivileged_protected_attributes']=}")

## Converting Our DataFrame to an AIF360 Dataset

The AI Fairness 360 (AIF360) toolkit provides different dataset classes to handle various types of data structures and use cases. Two commonly used classes are `BinaryLabelDataset` and `StandardDataset`. We will first use a BianryLabelDataset, which was the kind used in the tutorial example.




#### BinaryLabelDataset

**Purpose**: Designed specifically for binary classification tasks where the target variable (label) has only two possible outcomes.

**Key Features**:
- **Label Handling**: Assumes that the label (target variable) has binary values (e.g., 0 and 1, "yes" and "no").
- **Protected Attributes**: Handles protected attributes for fairness analysis.
- **Metrics Compatibility**: Provides compatibility with fairness metrics that are specific to binary classification tasks.
- **Examples**: Suitable for datasets like the Prisoner Recidivism dataset, where the task is to predict if the inmate was reincarcerated (binary Yes/No).

🔗[Link to Documentation](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.datasets.BinaryLabelDataset.html#aif360-datasets-binarylabeldataset)

In [ ]:
# Reviewing the structure of our DATA_DICT
DATA_DICT.keys()

In [ ]:
from aif360.datasets import BinaryLabelDataset

binary_dataset = BinaryLabelDataset(df=final_df,
                                    label_names=DATA_DICT['label_names'],
                                    protected_attribute_names=DATA_DICT['protected_attributes_names'],
                                    favorable_label = 0, # Non-recividism is favorable
                                    unfavorable_label = 1, # Recividism is unfavorable
                                    )
binary_dataset


#### Convenience Features of `BinaryLabelDataset`



1. **Simplified Initialization**:
   - **Default Configurations**: `BinaryLabelDataset` comes with default configurations optimized for binary classification, making it easier to set up for such tasks without needing extensive customization.

2. **Preconfigured Metrics Compatibility**:
   - **Metric Calculation**: When using `BinaryLabelDataset`, it’s straightforward to calculate fairness metrics specific to binary classification. These metrics, such as disparate impact or statistical parity difference, are preconfigured for binary outcomes.
   


In [ ]:
# # For loop (based on tutorial list comp) to create a list of dictionaries 
# # for unprivileged and privileged groups for each of the protected attributes
# unprivileged_groups = []
# privileged_groups = []

# for sens_ind, sens_attr in enumerate(protected_attributes_names):
    
#     unprivileged_groups.extend({sens_attr: v} for v in
#                        DATA_DICT['unprivileged_protected_attributes'][sens_ind])
    
#     privileged_groups.extend({sens_attr: v} for v in
#                      DATA_DICT['privileged_protected_attributes'][sens_ind])
    
# unprivileged_groups, privileged_groups

In [ ]:
DATA_DICT['unprivileged_protected_attributes'][0]

In [ ]:
DATA_DICT['protected_attributes_names'] ,DATA_DICT['privileged_protected_attributes'],DATA_DICT['unprivileged_protected_attributes']

# AI Fairness 360 Metrics

## Metric Classes

In order to calculate fairness metrics, we will leverage 2 classes from AIF360: BinaryLabelDatasetMetric and ClassificationMetric. The BinaryLabelDatasetMetric class is used to calculate the fairness metrics, while the ClassificationMetric class is used to calculate the classification metrics.

Additionally, the package includes a text explainer class, MetricTextExplainer, which can be used to explain the fairness metrics.


When instantiateing a meetric class, we must provide the list of privileged groups and unprivileged groups as a list of dictionaries.

```
### TROUBLESHOOTING `BinaryLabelDatasetMetric` CREATION
Unsure about parameters `privileged_groups` and `unprivileged_groups`.
- If i use the list of numpy arrays from DATA_DICT:
    - get error: `AttributeError: 'numpy.ndarray' object has no attribute 'items'`
- If i use the list of dictionaries from above (`unprivileged_groups`, `privileged_groups`) 
    - get error: `ValueError: 'privileged_groups' and 'unprivileged_groups' must be disjoint.
````

In [ ]:
# TEMP  testing of args - single dict per attribute

# For loop (based on tutorial list comp) to create a list of dictionaries 
# for unprivileged and privileged groups for each of the protected attributes
unprivileged_groups = []
privileged_groups = []


for sens_ind in range(len(protected_attributes_names)):
    # Saving the name of the current protected attribute
    sens_attr_name = protected_attributes_names[sens_ind]
    
    # Save the unique integer values for the current protected attribute
    unprivileged_groups.append({sens_attr_name: np.unique(DATA_DICT['unprivileged_protected_attributes'][sens_ind])})
    privileged_groups.append({sens_attr_name:  np.unique(DATA_DICT['privileged_protected_attributes'][sens_ind])})

unprivileged_groups, privileged_groups

In [ ]:
# ## Using single dict per class (instead of attribut)
# unprivileged_groups = []
# privileged_groups = []

# for send_ind, sens_attr in enumerate(protected_attributes_names):
#     unprivileged_groups.extend({sens_attr: v} for v in
#                        DATA_DICT['unprivileged_protected_attributes'][sens_ind])
    
#     privileged_groups.extend({sens_attr: v} for v in
#                      DATA_DICT['privileged_protected_attributes'][sens_ind])
    
# unprivileged_groups, privileged_groups

- BinaryLabelDatasetMetric class must be instantiated for 1 protected attribute at a time (e.g. Sex or Race, but not both).


In [ ]:
# Example:
from aif360.metrics import BinaryLabelDatasetMetric

# Calculating the disparate impact for Race
sens_ind = 0

# Printing the name of the sensitive attribute
print(f"{list(unprivileged_groups[sens_ind].keys())[0]}")

metric_race = BinaryLabelDatasetMetric(binary_dataset,
                                  privileged_groups=[privileged_groups[sens_ind]],
                                  unprivileged_groups=[unprivileged_groups[sens_ind]]
                                  )
metric_race

We will also instantiate a Metric TextExplainer for each class to help us interpret the metrics.

In [ ]:
# instantiate a metric text explainer using the BinaryLabelDatasetMetric object
explainer_race = MetricTextExplainer(metric_race)
explainer_race

We will make a second objecct for our second protected attribute: Sex


In [ ]:
# Calculating the disparate impact for sex
sens_ind = 1
print(f"{list(unprivileged_groups[sens_ind].keys())[0]}")

# Instantiate the metric class for the protected attribute.
metric_sex = BinaryLabelDatasetMetric(binary_dataset,
                                  privileged_groups=[privileged_groups[sens_ind]],
                                  unprivileged_groups=[unprivileged_groups[sens_ind]]
                                  )

# Instantiate the explainer for each protected attribute
explainer_sex = MetricTextExplainer(metric_sex)

## Fairness Metrics 

- We will discuss several AI Fairness 360 metrics in the next section. Some of the metrics include:
    - Disparate Impact
    - Mean Difference
    - Base Rate
    - Consistency
    - Statistical Parity Difference
    - Equal Opportunity Difference
    

### Disparate Impact

>**Disparate impact**, also known as **adverse impact** or **disparate treatment**, is a concept used in the field of employment and civil rights law to assess potential discrimination. It refers to a situation where a particular policy or practice, although neutral on its face, has a disproportionately negative impact on a protected group compared to other groups.

- In the context of fairness in machine learning, the disparate impact ratio is a fairness metric that measures the relative likelihood of a favorable outcome between different groups.
- It quantifies the difference in the probability of a positive outcome for the privileged group compared to the unprivileged group. 



**Interpreting Disparate Impact Ratio**
-  a disparate impact ratio of 1 indicates no disparity
-  a ratio less than 1 indicates a higher likelihood of a positive outcome for the unprivileged group
-  a ratio greater than 1 indicates a higher likelihood for the privileged group.

Assessing disparate impact is important to identify and address potential biases in machine learning models and ensure fairness in decision-making processes.



#### Sex

In [ ]:
# Calculate the disparate impact for sex
print(f"Disparate Impact (Sex): {metric_sex.disparate_impact()}")


# Exlpaining the disparate impact
print(explainer_sex.disparate_impact())


In the case of Sex, since the value is greater than 1 (1.126) the dataset is actually biased, with Females being more likely to be non-recidivists than male. We can see this for ourselves by looking at the cross tabulation the Sex and Reincarcerated features.

In [ ]:
crosstab = pd.crosstab(df['Sex'], df['Reincarcerated'],normalize='index')
ax = crosstab.plot(kind='bar')# stacked=True)
ax.set(ylabel='Proportion of Sex',title='Males (Sex=0) are more likely to be reincarcerated');

#### Race

In [ ]:
# Display the disparate impact for Race
print(f"Disparate Impact (Race): {metric_race.disparate_impact()}")
print(explainer_sex.disparate_impact())


In the case of Race, since the value is greater than 1 (1.05) the dataset is actually biased with non-whites being more likely to be non-recidivists than male.

In [ ]:
crosstab = pd.crosstab(df['Race'], df['Reincarcerated'],normalize='index')
ax = crosstab.plot(kind='bar')# stacked=True)
ax.set(ylabel='Proportion of Race',title='White (Race=0) are slighlty more likely to be reincarcerated');

### Mean Difference

> The **mean_difference** fairness metric is a measure used to assess the fairness of a machine learning model's predictions across different groups or demographics. It quantifies the average difference in predicted outcomes between different groups.

- In the context of fairness, groups can be defined based on sensitive attributes such as race, gender, or age. 
- The mean_difference metric calculates the average difference in predicted outcomes (e.g., probabilities or scores) between these groups.



To calculate the mean_difference fairness metric, you need to follow these steps:

1. Identify the sensitive attribute or group that you want to evaluate for fairness.
2. Split your dataset into subgroups based on the sensitive attribute.
3. For each subgroup, calculate the average predicted outcome.
4. Calculate the difference between the average predicted outcomes of different subgroups.
5. Take the absolute value of the differences to ensure that negative and positive differences are treated equally.
6, Finally, calculate the average of these absolute differences across all subgroups.

The resulting value represents the mean_difference fairness metric. 

>- A value close to zero indicates that the model's predictions are similar across different groups, suggesting fairness. On the other hand, a larger value indicates a significant difference in predictions between groups, suggesting potential bias or unfairness.


#### Sex

In [ ]:
print(f"Mean Difference (Sex): {metric_sex.mean_difference()}")
print(explainer_sex.mean_difference())

We can calculate this manually to confirm our understanding.

In [ ]:
# Calculating the mean difference manually
group_means = df.groupby("Sex")['Reincarcerated'].mean()
group_means

In [ ]:
# Subtract the mean of the unprivileged group from the privileged group
abs(group_means.loc[1.0] - group_means.loc[0.0])

We can see that the mean difference between groups is ~7.7%, which matches the metric class's results.

#### Race

In [ ]:
print(f"Mean Difference (Race): {metric_race.mean_difference()}")
print(explainer_race.mean_difference())

We can see that the mean difference between groups is ~3.4%.

Our dataset is more biased in terms of Sex than it is for Race. 

#### Base Rate

>The **base_rate** metric for fairness is a measure that evaluates the proportion of positive outcomes (e.g., approvals, acceptances, etc.) within a particular group. 
- It is used to compare the rates of favorable outcomes across different groups defined by sensitive attributes such as race, gender, or age.

To calculate the base_rate for a group, follow these steps:

1. Identify the group based on the sensitive attribute.
2. Count the number of positive outcomes (e.g., the number of approved loans) within the group.
3. Divide this count by the total number of instances in the group.

The formula for the base_rate is:

$ \text{Base Rate} = \frac{\text{Number of Positive Outcomes}}{\text{Total Number of Instances in the Group}} $

- The base_rate can then be compared across different groups to assess fairness. If the base_rate is significantly different between groups, it may indicate potential bias or unfairness in the decision-making process.


#### Sex

In [ ]:
## Base Rate
# print(explainer_train.base_rate())
priv_base_rate = metric_sex.base_rate()
unprov_base_rate = metric_sex.base_rate(privileged=False)

print("For protected attribute: Sex")
print(f"- Privileged Base Rate: {priv_base_rate}")
print(f"- Unprivileged Base Rate: {unprov_base_rate}")

In [ ]:
# ## Calculating manuallyi with pandas
# value_counts = df.groupby("Sex")['Reincarcerated'].value_counts(normalize=True)
# value_counts


In [ ]:
# Slice the base rate for the privileged group
value_counts.loc[0.0,0]

In [ ]:
# Slice the base rate for the unprivileged group
value_counts.loc[1.0,0]

We can see that the unpriviledged class for Sex is more likely to be a non-recividist (the desirable outcome).

#### Race

In [ ]:
## Base Rate
# print(explainer_train.base_rate())
priv_base_rate = metric_race.base_rate()
unprov_base_rate = metric_race.base_rate(privileged=False)

print("For protected attribute: Race")
print(f"- Privileged Base Rate: {priv_base_rate}")
print(f"- Unprivileged Base Rate: {unprov_base_rate}")


The base_rate metric is useful for identifying disparities in outcomes across different groups and is often used in conjunction with other fairness metrics to provide a comprehensive assessment of fairness in machine learning models.


#### Statistical Parity Difference

Statistical parity difference measures the difference in the probability of favorable outcomes between unprivileged and privileged groups.


In [ ]:
# Statistical Parity Difference for Sex
print(f"Statistical Parity Difference (Sex): {metric_sex.statistical_parity_difference()}")


In [ ]:
print(f"Statistical Parity Difference (Race): {metric_race.statistical_parity_difference()}")


Note that the values are equivalent to the previous mean_difference.


### Conclusion

In this blog post, we explored how to create dataset objects using both the `BinaryLabelDataset` class in AI Fairness 360. We also discussed some key fairness metrics to evaluate bias in datasets and models. By leveraging these tools, you can take a significant step towards building fairer and more ethical AI systems. In the next blog post, we will explore using AI Fairness 360 to asses the fairness of a machine learning model and the intervetions we can take to alleviate the bias. 


### Further Reading
- IBM AI Fairness 360: [AIF360 Documentation](https://aif360.mybluemix.net/)
- Fairness and Machine Learning by Solon Barocas, Moritz Hardt, and Arvind Narayanan
- Algorithmic Bias Detection and Mitigation: Best Practices and Policies to Reduce Consumer Harms by the Federal Trade Commission

### Call to Action
I encourage you to experiment with AI Fairness 360 in your projects and contribute to creating fair AI systems. Together, we can make AI work for everyone.

### References
- [Disparate Impact](https://en.wikipedia.org/wiki/Disparate_impact)
- [AI Fairness 360 GitHub Repository](https://github.com/IBM/AIF360)
- [Bias in AI: A Review of Literature](https://arxiv.org/abs/1908.09635)



___

## Glossary for AI Fairness 360 Terms

#### Protected Attributes
**Definition**: Attributes that refer to characteristics of individuals that are legally protected against discrimination. These can include race, gender, age, religion, disability status, and other similar attributes.
**Example**: In a dataset about job applicants, 'race' and 'gender' could be considered protected attributes.

#### Privileged Groups
**Definition**: Subsets of individuals within a dataset that have historically been favored or have had advantages in societal contexts based on certain protected attributes.
**Example**: In the context of gender, males may be considered a privileged group in certain employment datasets.

#### Unprivileged Groups
**Definition**: Subsets of individuals within a dataset that have historically been disadvantaged or have faced discrimination based on certain protected attributes.
**Example**: In the context of race, individuals identifying as Black or Hispanic might be considered unprivileged groups in certain societal contexts.

#### Favorable Label
**Definition**: The outcome or class in a dataset that is considered positive or beneficial for the individual.
**Example**: In a dataset predicting loan approvals, a favorable label would be 'approved' (indicating that the loan application was successful).

#### Unfavorable Label
**Definition**: The outcome or class in a dataset that is considered negative or detrimental for the individual.
**Example**: In a dataset predicting recidivism, an unfavorable label would be 'reoffended' (indicating that the individual reoffended after being released from prison).


___


# HOLD FOR BLOG POST 2:
- Consistency
- Equal Opportunity Difference


#### Equal Opportunity Difference

Equal opportunity difference measures the difference in true positive rates between unprivileged and privileged groups.


In [ ]:

from aif360.metrics import ClassificationMetric

# Assuming you have a model's predictions for demonstration purposes
# Example: true_labels and predicted_labels are arrays of true and predicted labels

true_labels = binary_dataset.labels
predicted_labels = binary_dataset.labels  # Placeholder, replace with actual model predictions

# Create a ClassificationMetric object
classification_metric = ClassificationMetric(
    binary_dataset,
    binary_dataset,  # Use the same dataset for demonstration
    unprivileged_groups=[{'Race': 0}],
    privileged_groups=[{'Race': 1}]
)

# Calculate Equal Opportunity Difference
print(f"Equal Opportunity Difference: {classification_metric.equal_opportunity_difference()}")


#### Consistency

> The consistency fairness metric measures how consistent the predictions of a machine learning model are for similar instances. It evaluates whether similar individuals receive similar predictions, which is an important aspect of fairness.

To calculate the consistency metric, follow these steps:

1. Identify Similar Instances: For each instance in the dataset, identify a set of similar instances. 
    - This can be done using a distance metric (e.g., Euclidean distance) in the feature space. 
    - The number of similar instances to consider is typically specified by a parameter like n_neighbors.

2. Calculate Prediction Differences: For each instance, calculate the difference between its predicted outcome and the predicted outcomes of its similar instances.

3. Average the Differences: Compute the average of these differences across all instances in the dataset.

The formula for the consistency metric can be expressed as:

$\text{Consistency} = 1 - \frac{1}{N} \sum_{i=1}^{N} \frac{1}{k} \sum_{j \in \text{Neighbors}(i)} | \hat{y}_i - \hat{y}_j | $

Where:

- $ N $ is the total number of instances.
- $ k $ is the number of neighbors (similar instances) considered.
- $ \hat{y}_i$ is the predicted outcome for instance $ i $.
- $ \hat{y}_j $ is the predicted outcome for the $ j $-th neighbor of instance $ i $.


> A higher consistency value indicates that the model's predictions are more consistent for similar instances, suggesting a fairer model.


In [ ]:
## Consistency
print(explainer_race.consistency())

explainer_race.consistency(n_neighbors=5)

In [ ]:
## Consistency
print(explainer_sex.consistency())

explainer_sex.consistency(n_neighbors=5)

### Modeling Metrics

> Add explanation about need for a model/predictions for final metrics

In [ ]:
# Train/test/split with StandardDataset

train, test = binary_dataset.split([0.7], shuffle=True)
train.features.shape, test.features.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import ConfusionMatrixDisplay, classification_report

In [ ]:
clf = RandomForestClassifier(n_estimators=200,
                             max_depth=12,
                             min_samples_leaf=2,
                             random_state=42, class_weight='balanced')
clf.fit(train.features, train.labels.ravel())

# Evaluate model
y_hat_test = clf.predict(test.features)


def evaluate_model(y_true, y_pred):
    """Minimal evaluation of a model's performance."""
    print(classification_report(y_true.labels, y_pred))
    ConfusionMatrixDisplay.from_predictions(y_true.labels, y_pred,
                                            normalize='true',cmap='Greens',
                                            display_labels=['Non-Recid','Recid']);
    
evaluate_model(test, y_hat_test)

In [ ]:
type(train), type(test), type(y_hat_test)

In [ ]:
from aif360.metrics import ClassificationMetric
from aif360.datasets import BinaryLabelDataset

# Create a ClassificationMetric object
classification_metric = ClassificationMetric(test, test,
                                             privileged_groups=privileged_groups,
                                             unprivileged_groups=unprivileged_groups)
                                             

# Calculate consistency
consistency_score = classification_metric.consistency(n_neighbors=5)
print(f"Consistency: {consistency_score}")

In [ ]:
# Calculate Equal Opportunity Difference
print(f"Equal Opportunity Difference: {classification_metric.equal_opportunity_difference()}")


#### Statistical Parity Difference

Statistical parity difference measures the difference in the probability of favorable outcomes between unprivileged and privileged groups.


In [ ]:

# Statistical Parity Difference for BinaryLabelDataset
# print(f"Statistical Parity Difference (BinaryLabelDataset): {binary_metric.statistical_parity_difference()}")

# Statistical Parity Difference for StandardDataset
print(f"Statistical Parity Difference (StandardDataset): {standard_metric.statistical_parity_difference()}")


#### Equal Opportunity Difference

Equal opportunity difference measures the difference in true positive rates between unprivileged and privileged groups.


In [ ]:

from aif360.metrics import ClassificationMetric

# Assuming you have a model's predictions for demonstration purposes
# Example: true_labels and predicted_labels are arrays of true and predicted labels

true_labels = binary_dataset.labels
predicted_labels = binary_dataset.labels  # Placeholder, replace with actual model predictions

# Create a ClassificationMetric object
classification_metric = ClassificationMetric(
    binary_dataset,
    binary_dataset,  # Use the same dataset for demonstration
    unprivileged_groups=[{'race': 0}],
    privileged_groups=[{'race': 1}]
)

# Calculate Equal Opportunity Difference
print(f"Equal Opportunity Difference: {classification_metric.equal_opportunity_difference()}")



### Additional Similar Terms for AIF360

#### Bias Mitigation
**Definition**: The process of reducing or eliminating bias in machine learning models. This can involve techniques and algorithms that adjust the data or model to achieve fairer outcomes.
**Example**: Reweighing, which adjusts the weights of different groups in the training data, is a bias mitigation technique.

#### Disparate Impact
**Definition**: A measure used to determine if a decision-making process disproportionately affects a particular protected group. It is calculated as the ratio of the rate of a favorable outcome for the unprivileged group to the rate of a favorable outcome for the privileged group.
**Example**: If the hiring rate for women (unprivileged group) is 30% and for men (privileged group) is 60%, the disparate impact is 0.5 (30% / 60%).

#### Fairness Metrics
**Definition**: Quantitative measures used to assess the fairness of a machine learning model. These metrics evaluate how equally outcomes are distributed among different groups based on protected attributes.
**Example**: Statistical parity, equal opportunity difference, and disparate impact are examples of fairness metrics.

#### Bias Detection
**Definition**: The process of identifying bias in datasets or machine learning models. This involves analyzing the data and model predictions to uncover patterns of unfair treatment based on protected attributes.
**Example**: Calculating the disparate impact ratio to check if a model's predictions are biased against a particular racial group.

#### Adversarial Debiasing
**Definition**: A bias mitigation technique where an adversarial model is trained to predict the protected attribute from the original model’s predictions. The goal is to adjust the original model to make it difficult for the adversarial model to correctly predict the protected attribute, thus reducing bias.
**Example**: Training an adversarial network alongside the main model to ensure that the predictions are not correlated with the protected attributes.

---

This glossary should help your readers understand the key concepts and terminology associated with AI Fairness 360. If you have any more terms to include or need further details, let me know!

# APPENDIX


3. **Assumptions for Binary Labels**:
   - **Favorable and Unfavorable Classes**: `BinaryLabelDataset` explicitly handles the designation of favorable and unfavorable classes, streamlining tasks where such distinctions are important.
   - **Example**:
     ```python
     dataset = BinaryLabelDataset(df, label_name='income', favorable_classes=[1], protected_attribute_names=['race'])
     ```



### StandardDataset



**Purpose**: A more general-purpose dataset class that can handle a wide variety of data structures, including both binary and multi-class classification tasks.

**Key Features**:
- **Label Handling**: Can handle binary, multi-class, and continuous labels.
- **Flexibility**: More flexible in terms of the types of data it can represent.
- **Protected Attributes**: Handles protected attributes for fairness analysis.
- **Customizability**: Provides more options for customization, including handling multiple protected attributes and various types of labels.
- **Examples**: Suitable for datasets like the COMPAS dataset (recidivism prediction), where the task could involve multiple classes or continuous labels.


In [ ]:

from aif360.datasets import StandardDataset

# Create a StandardDataset
standard_dataset = StandardDataset(
    final_df,
    label_name=DATA_DICT['label_names'][0], # Recidivism is the label
    favorable_classes=[0], # Non-recidivism is favorable
    protected_attribute_names=DATA_DICT['protected_attributes_names'], #['Race','Sex'],  # List of the protected attributes
    privileged_classes=DATA_DICT["privileged_protected_attributes"],#[[race_map['White']],[sex_map['Male']]]  # Privileged group: White
    # unprivileged_classes = DATA_DICT["unprivileged_protected_attributes"]
)

# Print dataset details
print(standard_dataset.feature_names)
print(standard_dataset.label_names)
print(standard_dataset.protected_attribute_names)


In [ ]:
# Example:
from aif360.metrics import BinaryLabelDatasetMetric

sens_ind = 0
metric = BinaryLabelDatasetMetric(standard_dataset,
                                  privileged_groups=[privileged_groups[sens_ind]],
                                  unprivileged_groups=[unprivileged_groups[sens_ind]]
                                  )

print(f"Disparate Impact: {metric.disparate_impact()}")


In [ ]:

# from aif360.datasets import StandardDataset

# # Create a StandardDataset
# standard_dataset = StandardDataset(
#     final_df,
#     label_name='Reincarcerated', # Recidivism is the label
#     favorable_classes=[0], # Non-recidivism is favorable
#     protected_attribute_names=['Race','Sex'],  # List of the protected attributes
#     privileged_classes=[[race_map['White']],[sex_map['Male']]]  # Privileged group: White
# )

# # Print dataset details
# print(standard_dataset.feature_names)
# print(standard_dataset.label_names)
# print(standard_dataset.protected_attribute_names)


___

### Comparison of BinaryLabel vs. Standard Datasets

#### BinaryLabelDataset


**Example Usage**:
```python
from aif360.datasets import BinaryLabelDataset

dataset = BinaryLabelDataset(df, label_name='income', favorable_classes=[1], protected_attribute_names=['race'])
```



**Advantages**:
- Simplified handling of binary labels.
- Directly compatible with binary classification metrics in AIF360.

**Limitations**:
- Limited to binary classification tasks.
- Requires preprocessing if the dataset has multi-class labels or other complexities.


#### StandardDataset


**Example Usage**:
```python
from aif360.datasets import StandardDataset

dataset = StandardDataset(df, label_name='recidivism', favorable_classes=[0], protected_attribute_names=['race'], privileged_classes=[[1]])
```

**Advantages**:
- Greater flexibility and customizability.
- Suitable for a broader range of tasks beyond binary classification, including multi-class and regression tasks.
- Allows for more complex preprocessing and handling of different types of labels.

**Limitations**:
- More complex to set up and use compared to `BinaryLabelDataset`.
- May require additional customization and configuration for specific use cases.



#### Key Differences

1. **Target Use Case**:
   - `BinaryLabelDataset`: Specifically designed for binary classification tasks.
   - `StandardDataset`: General-purpose, suitable for binary, multi-class, and continuous label tasks.

2. **Label Handling**:
   - `BinaryLabelDataset`: Assumes binary labels.
   - `StandardDataset`: Can handle binary, multi-class, and continuous labels.

3. **Flexibility**:
   - `BinaryLabelDataset`: Less flexible, optimized for simplicity in binary classification.
   - `StandardDataset`: More flexible, can handle complex datasets with various types of labels and multiple protected attributes.

4. **Setup and Configuration**:
   - `BinaryLabelDataset`: Easier and quicker to set up for binary classification tasks.
   - `StandardDataset`: Requires more setup and customization but offers greater versatility.



#### ~~Conclusion~~

The choice between `BinaryLabelDataset` and `StandardDataset` depends on the specific requirements of your project. If you are working on a straightforward binary classification task, `BinaryLabelDataset` provides a simpler and more streamlined option. However, if your project involves more complex data structures, such as multi-class labels or multiple protected attributes, `StandardDataset` offers the flexibility and customizability needed to handle these complexities effectively.
___

In [ ]:
from aif360.datasets import StandardDataset, BinaryLabelDataset

# Assuming 'label' is your target column and 'gender' is your protected attribute

binary_dataset = BinaryLabelDataset(
        df=final_df,
        label_names=["Reincarcerated"],
        protected_attribute_names = protected_attributes,
        # privileged_groups = privileged_groups,
        # favorable_label=[{"Race":0],
        # unfavorable_label=1
    )
binary_dataset
from aif360.datasets import BinaryLabelDataset

# Create a BinaryLabelDataset with multiple protected attributes
binary_dataset = BinaryLabelDataset(
    df,
    label_name='recidivism',
    favorable_classes=[0],  # Non-recidivism is favorable
    protected_attribute_names=['race', 'gender'],
    privileged_classes=[[1], [1]]  # Privileged groups: White and Male
)

# Print dataset details
print(binary_dataset.feature_names)
print(binary_dataset.label_names)
print(binary_dataset.protected_attribute_names)

# target_col = "Reincarcerated"
# # Creating a data dictionary like the one used in AIF360's medical expenditure example
# data_dict={'feature_names':df.drop(coluns=target_col).columns,  # Question: Use transofrmed or original??
#            'label_names':[target_col], 
#            'protected_attribute_names': ["Sex"],
#            'privileged_protected_attributes': [np.array([1])], 
#            'unprivileged_protected_attributes': [np.array([0])],
#            }


The `BinaryLabelDataset` class in AI Fairness 360 is a specialized subclass of `StandardDataset` designed specifically for binary classification tasks. Because `BinaryLabelDataset` is built on top of `StandardDataset`, it inherits all its functionalities and adds some specific conveniences for handling binary labels. Therefore, there is nothing inherently unique that `BinaryLabelDataset` can do that `StandardDataset` cannot. However, `BinaryLabelDataset` provides some convenience features and simplified handling specifically for binary classification tasks. Here’s a detailed look:



### Capabilities of `StandardDataset`

`StandardDataset` is more general and can handle a variety of data structures, including:

1. **Multi-Class Classification**:
   - **Multiple Labels**: `StandardDataset` can be used for tasks involving more than two classes, offering greater flexibility for datasets with multi-class labels.
   - **Example**:
     ```python
     from aif360.datasets import StandardDataset

     dataset = StandardDataset(df, label_name='health_status', favorable_classes=['healthy'], protected_attribute_names=['race'])
     ```

2. **Continuous Labels**:
   - **Regression Tasks**: `StandardDataset` can handle datasets where the target variable is continuous, making it suitable for regression tasks.
   - **Example**:
     ```python
     dataset = StandardDataset(df, label_name='income', favorable_classes=[lambda x: x > 50000], protected_attribute_names=['race'])
     ```

3. **Custom Preprocessing**:
   - **Flexible Configuration**: `StandardDataset` allows for more complex preprocessing and configuration, accommodating a wide range of use cases beyond binary classification.

### Conclusion

While `BinaryLabelDataset` offers a streamlined, convenient interface specifically for binary classification tasks, anything it can do can also be achieved with `StandardDataset` through additional customization. The main advantage of using `BinaryLabelDataset` lies in its ease of use and the fact that it simplifies certain aspects of handling binary labels, such as setting up fairness metrics and defining favorable/unfavorable classes. However, for more complex tasks or different types of labels, `StandardDataset` is the more versatile and flexible option.

Now that we've successfuly reproduced the structure of the original data dictionary, we have the information we need to follow the tutorial's example code.

In [ ]:
# print(sex_map)
# [{"Sex":v} for k,v in sex_map.items() if not k.lower().startswith('m')]

### Binary Dataset

Below is a detailed comparison and contrast of these two classes.

**BinaryLabelDataset**

Purpose: Designed specifically for binary classification tasks where the target variable (label) has only two possible outcomes.

Key Features:

* Label Handling: Assumes that the label (target variable) has binary values (e.g., 0 and 1, “yes” and “no”).
* Protected Attributes: Handles protected attributes for fairness analysis.
* Metrics Compatibility: Provides compatibility with fairness metrics that are specific to binary classification tasks.
* Examples: Suitable for datasets like the Adult Income dataset, where the task is to predict if income is above or below $50K.


### Creating a `StandardDataset`

The `StandardDataset` is more flexible and can handle binary, multi-class, and continuous labels.


#### NOTE ON UPDATING data_dict and unprivileged_groups/privileged_groups

>- I think data_dict needs to have...
>    - Just numbers in arrays?

### BOOKMARK: CALL WITH BRENDA 07/25/24


In [ ]:
# ### BOOKMARK: CALL WITH BRENDA 07/25/24
# ### alt way to create priv and unpriv groups
# prot_attrs = ['Race','Sex']
# privileged_protected_attributes = [[0],[0]]

# protected_attr_maps = {"Race":race_map,
#                  "Sex": sex_map}

# # unprivileged_protected_attributes = []/
# unprivileged_groups = []
# for idx, protected_attr in enumerate(prot_attrs):
#     temp_unpriv = []
#     for k,v in protected_attr_maps[protected_attr].items():
#         if v not in privileged_protected_attributes[idx]:#[0]:
#             temp_unpriv.append(v)
#         # privileged_protected_attributes.append(v_)
#         # else:
#     if len(temp_unpriv) > 0:
#         unprivileged_groups.append( np.array(temp_unpriv)
#         # unprivileged_protected_attributes.append({protected_attr:temp_unpriv})
#         # unprivileged_protected_attributes.append({protected_attr:v})
# unprivileged_groups
    

In [ ]:
# from pprint import pprint
# pprint(data_dict)

In [ ]:


# # # Define the unprivileged and privileged groups
# privileged_groups =[ ]
# unprivileged_groups = [ ]

# for protected_attr in data_dict['privileged_protected_attributes']:
#     privileged_groups.append(protected_attr)
# for protected_attr in data_dict['unprivileged_protected_attributes']:
#     unprivileged_groups.append(protected_attr)

# privileged_groups, unprivileged_groups

# # for protected_attr in data_dict['protected_attribute_names']:
    
# #     temp = data_dict['privileged_protected_attributes']
# #     privileged_groups.append({protected_attr: temp[0][0]})
    
# #     temp = data_dict['unprivileged_protected_attributes']
# #     unprivileged_groups.append({protected_attr: temp[0][0]})
# # privileged_groups, unprivileged_groups




### Exploring Fairness Metrics

AI Fairness 360 provides several fairness metrics to evaluate bias in datasets and models. We'll explore some key metrics using both `BinaryLabelDataset` and `StandardDataset`.

#### Disparate Impact

Disparate impact is a ratio that measures the relative likelihood of a favorable outcome between unprivileged and privileged groups.


In [ ]:

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.explainers import MetricTextExplainer


In [ ]:

# # Disparate Impact for BinaryLabelDataset
# binary_metric = BinaryLabelDatasetMetric(
#     binary_dataset,
#     privileged_groups=[{'Race': 0}],
#     unprivileged_groups=[{"Race":i} for i in range(1,len(race_map))]
# )
# print(f"Disparate Impact (BinaryLabelDataset): {binary_metric.disparate_impact()}")


In [ ]:
# priv_set = set(privileged_groups)
# unpriv_set = set(unprivileged_groups)
# priv_set.disjoint(unpriv_set)

In [ ]:

# Disparate Impact for StandardDataset
standard_metric_race = BinaryLabelDatasetMetric(
    standard_dataset,
    privileged_groups=privileged_groups,#[{'Race': race_map['White']}], # White
    unprivileged_groups=unprivileged_groups#[{"Race":i} for race,i in race_map.items() if "White" not in race] # All other
)



### Creating a `BinaryLabelDataset`


To calculate ClassificationMetrics we need to use a special variant of the StandardDataset - the BinaryLabel Dataset.
`TO DO: ADD BRIEF EXPLAINATION RE: DIFFERENCE BETWEEN STANDARD AND BINARY LABEL DATASETS`


The `BinaryLabelDataset` is tailored for binary classification tasks where the target variable has two possible outcomes.


In [ ]:
# from aif360.datasets import BinaryLabelDataset

# Create a BinaryLabelDataset
binary_dataset = BinaryLabelDataset(
    df = final_df,
    label_names=['Reincarcerated'],
    # favorable_classes=[0],  # Non-recidivism is favorable
    favorable_label=0,  # Non-recidivism is favorable
    unfavorable_label=1, # Recidivism is unfavorable
    protected_attribute_names=['Race','Sex'],
    # privileged_classes=[[0],[0]]  # Privileged group: White
)

# Print dataset details
print(binary_dataset.feature_names)
print(binary_dataset.label_names)
print(binary_dataset.protected_attribute_names)



### TO DO: Need to add a model for the metrics below

# OLD


### Adding Multiple Protected Attributes



Here’s how you can specify multiple protected attributes for both `BinaryLabelDataset` and `StandardDataset`.



#### Example with `BinaryLabelDataset`



Let's say you want to include both `race` and `gender` as protected attributes:



```python
from aif360.datasets import BinaryLabelDataset

# Encode 'race' and 'gender'
df['race'] = df['race'].map({'Black': 0, 'White': 1, 'Other': 2})
df['gender'] = df['gender'].map({'Male': 1, 'Female': 0})

# Create a BinaryLabelDataset with multiple protected attributes
binary_dataset = BinaryLabelDataset(
    df,
    label_name='recidivism',
    favorable_classes=[0],  # Non-recidivism is favorable
    protected_attribute_names=['race', 'gender'],
    privileged_classes=[[1], [1]]  # Privileged groups: White and Male
)

# Print dataset details
print(binary_dataset.feature_names)
print(binary_dataset.label_names)
print(binary_dataset.protected_attribute_names)
```



#### Example with `StandardDataset`

Similarly, for `StandardDataset`, you can specify multiple protected attributes:

```python
from aif360.datasets import StandardDataset

# Create a StandardDataset with multiple protected attributes
standard_dataset = StandardDataset(
    df,
    label_name='recidivism',
    favorable_classes=[0],
    protected_attribute_names=['race', 'gender'],
    privileged_classes=[[1], [1]]  # Privileged groups: White and Male
)

# Print dataset details
print(standard_dataset.feature_names)
print(standard_dataset.label_names)
print(standard_dataset.protected_attribute_names)
```

### Exploring Fairness Metrics with Multiple Protected Attributes

When using multiple protected attributes, you can still calculate fairness metrics. Here’s how you can do it:

#### Disparate Impact

Calculate disparate impact considering multiple protected attributes:

```python
from aif360.metrics import BinaryLabelDatasetMetric

# Disparate Impact for BinaryLabelDataset
binary_metric = BinaryLabelDatasetMetric(
    binary_dataset,
    privileged_groups=[{'race': 1, 'gender': 1}],
    unprivileged_groups=[{'race': 0, 'gender': 0}]
)
print(f"Disparate Impact (BinaryLabelDataset): {binary_metric.disparate_impact()}")

# Disparate Impact for StandardDataset
standard_metric = BinaryLabelDatasetMetric(
    standard_dataset,
    privileged_groups=[{'race': 1, 'gender': 1}],
    unprivileged_groups=[{'race': 0, 'gender': 0}]
)
print(f"Disparate Impact (StandardDataset): {standard_metric.disparate_impact()}")
```

#### Statistical Parity Difference

Calculate statistical parity difference considering multiple protected attributes:

```python
# Statistical Parity Difference for BinaryLabelDataset
print(f"Statistical Parity Difference (BinaryLabelDataset): {binary_metric.statistical_parity_difference()}")

# Statistical Parity Difference for StandardDataset
print(f"Statistical Parity Difference (StandardDataset): {standard_metric.statistical_parity_difference()}")
```

#### Equal Opportunity Difference

Calculate equal opportunity difference considering multiple protected attributes:

```python
from aif360.metrics import ClassificationMetric

# Assuming you have a model's predictions for demonstration purposes
true_labels = binary_dataset.labels
predicted_labels = binary_dataset.labels  # Placeholder, replace with actual model predictions

# Create a ClassificationMetric object
classification_metric = ClassificationMetric(
    binary_dataset,
    binary_dataset,  # Use the same dataset for demonstration
    unprivileged_groups=[{'race': 0, 'gender': 0}],
    privileged_groups=[{'race': 1, 'gender': 1}]
)

# Calculate Equal Opportunity Difference
print(f"Equal Opportunity Difference: {classification_metric.equal_opportunity_difference()}")
```



### Conclusion

In summary, AI Fairness 360 allows you to handle multiple protected attributes in both `BinaryLabelDataset` and `StandardDataset`. This capability is crucial for analyzing and mitigating bias across various dimensions, ensuring a more comprehensive approach to fairness in AI systems.

Feel free to expand on any sections or let me know if you need further details or specific explanations!

# BLOG V1

## Title: **AI Fairness 360: Ensuring Equity in Machine Learning**


### 👉Defining the Protected Attributes and Priveledged/Unpriveldged Groups

In [ ]:
# # Convert the DataFrame to an AIF360-compatible dataset
# protected_attributes= [ 'Race','Sex']
# privileged_groups = [{'Race': 0}, {"Sex":0}]
# unprivileged_groups = [{"Race":i} for i in range(1,len(race_map))] + [{"Sex":1}]
# unprivileged_groups

# # # Convert the DataFrame to an AIF360-compatible dataset
# # protected_attributes= [ 'Sex']
# # privileged_groups = [{"Sex":0}]
# # unprivileged_groups = [{"Sex":1}]
# # unprivileged_groups

#### Construct the AIF360 Dataset

In [ ]:
# # Remaining categorical features must be encoded
# from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder(drop=None, sparse_output=False)

# # Drop unnecessary columns 
# drop_cols = ['Supervising Unit','Supervision Start Date','Supervision End Date']

# # Categorical columns to be encoded
# cat_cols=  ['Supervision Type','Supervision End Reason',"Supervision End Reason",'Supervision Offense Class','Supervision Offense Type',
#        'Supervision Offense Subtype']


# ohe_df = encoder.fit_transform(df[cat_cols])

# # Concatenate one-hot encoded columns with the original DataFrame
# # final_df = pd.concat([df.drop(columns=[*cat_cols, *drop_cols]), ohe_df],axis=1)
# final_df = pd.get_dummies(df.drop(columns=drop_cols), columns=cat_cols)
# final_df

In [ ]:
# final_df.info()

In [ ]:
# obj_cols = final_df.select_dtypes('object').columns
# obj_cols

In [ ]:
# final_df[obj_cols] = final_df[obj_cols].astype(float)

In [ ]:
# df.head()
# df.info()

In [ ]:
# # Define the unprivileged and privileged groups
# privileged_groups =[ ]
# unprivileged_groups = [ ]

# for protected_attr in data_dict['protected_attribute_names']:
    
#     temp = data_dict['privileged_protected_attributes']
#     privileged_groups.append({protected_attr: temp[0][0]})
    
#     temp = data_dict['unprivileged_protected_attributes']
#     unprivileged_groups.append({protected_attr: temp[0][0]})
# privileged_groups, unprivileged_groups

# NEW BLOG POST: Detecting and Mitigatin Bias?

> NEED NEW INTRO


**Exploring the Dataset**
Before diving into bias detection, it's important to understand the dataset. The Adult Income dataset includes features such as age, education, occupation, race, and gender.



```python
print(dataset.features)
print(dataset.labels)
```



**Detecting Bias**
Next, we will use AI Fairness 360 to detect bias in the dataset. We will start by calculating the disparate impact, a commonly used fairness metric.



```python
from aif360.metrics import BinaryLabelDatasetMetric

metric = BinaryLabelDatasetMetric(dataset, privileged_groups=[{'race': 1}], unprivileged_groups=[{'race': 0}])
print(f"Disparate Impact: {metric.disparate_impact()}")
```



**Mitigating Bias**
After detecting bias, the next step is to mitigate it. We will use the reweighing algorithm to adjust the weights of different groups in the dataset to ensure fairer outcomes.



```python
from aif360.algorithms.preprocessing import Reweighing

rw = Reweighing(unprivileged_groups=[{'race': 0}], privileged_groups=[{'race': 1}])
dataset_transf = rw.fit_transform(dataset)
```



**Evaluating Results**
Finally, we will evaluate the effectiveness of our bias mitigation efforts by comparing the disparate impact metric before and after applying the reweighing algorithm.

```python
metric_transf = BinaryLabelDatasetMetric(dataset_transf, privileged_groups=[{'race': 1}], unprivileged_groups=[{'race': 0}])
print(f"Disparate Impact after mitigation: {metric_transf.disparate_impact()}")
```



### Conclusion

In this lesson, we explored the importance of fairness in AI and how the AI Fairness 360 toolkit can help detect and mitigate bias in machine learning models. By ensuring our AI systems operate fairly, we can create more equitable and just outcomes for all.



### Further Reading
- IBM AI Fairness 360: [AIF360 Documentation](https://aif360.mybluemix.net/)
- Fairness and Machine Learning by Solon Barocas, Moritz Hardt, and Arvind Narayanan
- Algorithmic Bias Detection and Mitigation: Best Practices and Policies to Reduce Consumer Harms by the Federal Trade Commission

### Call to Action
I encourage you to experiment with AI Fairness 360 in your projects and contribute to creating fair AI systems. Together, we can make AI work for everyone.

### References
- [Disparate Impact](https://en.wikipedia.org/wiki/Disparate_impact)
- [AI Fairness 360 GitHub Repository](https://github.com/IBM/AIF360)
- [Bias in AI: A Review of Literature](https://arxiv.org/abs/1908.09635)


# APPENDIX

## Old way of preparing data dict and groups

In [ ]:
# # Recreateing the privileged_groups to match the tutorial example
# # Make new mapping of group integers -> group names
# inverse_map = {v:k for k,v in current_attr_map_dict.items()}

# # Empoty List to save integer arrays of protected_attributes
# current_priviledged_protected_attributes = []
# current_unpriviledged_protected_attributes = []
 
# for group_num in unique_groups:
#     # Get the name of the current group
#     group_name = inverse_map[group_num]
    
#     # If the group_name is in list of priviledged groups, append to the priviledged list
#     if group_name in current_attr_priv_group_names:
#         current_priviledged_protected_attributes.append(group_num)
        
#     # Else, append to the unprivileged list    
#     else:
#         current_unpriviledged_protected_attributes.append(group_num)

# ## Convert the lists to numpy arrays
# current_priviledged_protected_attributes = np.array(current_priviledged_protected_attributes)
# current_unpriviledged_protected_attributes = np.array(current_unpriviledged_protected_attributes)
# current_priviledged_protected_attributes, current_unpriviledged_protected_attributes

In [ ]:

    
# current_privileged_prot_attrs = [np.array(current_attr_map_dict['White'])]
# current_unprivileged_prot_attrs = [{current_attr: v} for v in unique_groups if v not in current_privileged_groups]

# current_privileged_protected_attributes

In [ ]:
# # Set the unprivileged_groups
# current_unprivileged_groups = [{current_attr: v} for v in unique_groups if v not in current_privileged_groups]
# current_privileged_groups

In [ ]:
 
# data_dict={'feature_names':feature_names
#            'label_names':label_names, 
#            'protected_attribute_names': ["Race","Sex"],
#            'privileged_protected_attributes': [{'Race':race_map['White']},{"Sex":sex_map['Male']}] , #[np.array([1])], 
#            'unprivileged_protected_attributes': [{"Race":v} for k,v in race_map.items() if "white" not in k.lower()] + 
#                [{"Sex":v} for k,v in sex_map.items() if not k.lower().startswith('m')]
                
#            }


In [ ]:
# # Define the unprivileged and privileged groups
# privileged_groups =[ ]
# unprivileged_groups = [ ]

# # For eeach protected attribute, define the privileged and unprivileged groups
# for protected_attr in data_dict['protected_attribute_names']:
    
#     temp = data_dict['privileged_protected_attributes']
#     privileged_groups.append({protected_attr: temp[0][0]})
    
#     temp = data_dict['unprivileged_protected_attributes']
#     unprivileged_groups.append({protected_attr: temp[0][0]})
# privileged_groups, unprivileged_groups